#### Imports :

In [1]:
import json
import os
import random
import spacy

from errorTyping import genErrorTyping
from errorPhoneme import genErrorPhoneme
from inversion import genInversionLetterKeypad
from oversight import genOversightLetter

#### [ ! ] Variables (toujours vérifier avant lancement du _notebook_) :

In [2]:
nlp = spacy.load('fr_fasttext_lg')
nameOfHealthyCorpus = 'inputs/wikiTest'
# nameOfCorpusHealthyAndSickWordCorpus = 'compromisedCorpus.csv'
nameOfCorpusHealthyAndSickWordCorpus = 'results/CC-WikiTest.txt'
errorLigneGenCorpus = 'results/testDeGen.txt'

resultFile = open(nameOfCorpusHealthyAndSickWordCorpus, "a")

#### Méthodes :

In [3]:
def choiceConfigKeyboard():
    repository = "disposition_clavier/"                 # repertoire où réside les fichiers dictionnaires
    listFileName = os.listdir(repository)               # fonction permettant de lister l'ensemble des fichiers dans le repertoire
    print("This keyboard config are available :")
    i = 0

    for fileName in listFileName:                       # pour chaque nom de fichier présent dans la liste des fichiers listé précèdement
        indice = str(i)
        print("[" + indice + "]: " + fileName)          # affichage de ces fichiers avec leur numéro dans la liste
        i += 1

    choice = input("Thanks to choose keyboard disposition: [default: 0] ")

    if choice in ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9') and isinstance(choice, int) is False:   # en admettant que la liste ne peux etre plus grande que 9
        chc = int(choice)                               # formate le choix de l'utilisateur en INT
        choiceKeyboard = listFileName[chc]
        return choiceKeyboard                           # en fonction du choix donné par le user, affichage du fichier concerné
    if choice == '':
        choiceKeyboard = listFileName[0]                # si vide, alors application du choix par défaut : 0, donc le premier de la liste
        return choiceKeyboard
    else:
        print("It's not what is expected ! retry ...")  # si cette condition est rempli c'est que l'utilmisateur a tenté de rentrer un caractère nom attendu
        choiceConfigKeyboard()                          # on lui demande donc de reformuler sa saisie

In [4]:
def splitText(text):
    tmpTextForTreatment = text.split(' ')
    return tmpTextForTreatment

In [5]:
def freqWordWithMist():
    nWord = input("\nWhat will be the frequency of words with mistakes ?\n[min: 1; max: 9; default: 1] ")
    print("\n")

    if nWord == "":
        nWord = 1
        return nWord
    if nWord not in ('1', '2', '3', '4', '5', '6', '7', '8', '9'):
        print("Input value incorrect. Must be integer or in the range given !")
        freqWordWithMist()
    else:
        return nWord

In [6]:
def randomChoiceinTblOfWord(tblOfWrd):
    # print("with love")
    # print("\nMot dans le tableau :", tblOfWrd)
    # print("Longueur du tableau : ", tblOfWrd.__len__())
    r = random.randint(0, tblOfWrd.__len__() - 1)
    # print("Random :", r)
    # print(tblOfWrd[r])
    return tblOfWrd[r]

In [79]:
def generateErrorInWord(kbrd, frequency, text):
    
    print('entrée: ', text)
    free = False
    
    for indice in range(0, text.__len__(), frequency):

        r = random.randint(1, 4)
        print('r : ', r)
        tblOfWordReturn = []
        print('mot en cours : ', text[indice])
            
        switch = {
            1: genErrorTyping(nlp, kbrd, text[indice]),
            2: genInversionLetterKeypad(nlp, text[indice]),
            3: genOversightLetter(nlp, text[indice]),
            4: genErrorPhoneme(nlp, text[indice])
        }
        tblOfWordReturn=switch.get(r)
        
        if tblOfWordReturn == [''] or not tblOfWordReturn:
            text[indice] = text[indice]
        else:
            text[indice] = randomChoiceinTblOfWord(tblOfWordReturn)
            free = True

    if free == True:
        textToShow = " ".join(text)
        print('textToShow : ', textToShow)
        text = []
        return textToShow
    
    else:
        print('No change noted')
        textToShow = generateErrorInWord(kbrd, frequency, text)
        return textToShow

#### Envoi en sauvegarde :

In [80]:
def saveResult(idOfHealthyText, healhyText, cancerText):

    ###resultFile.write(idOfHealthyText)
    ###resultFile.write("µ")
    resultFile.write(cancerText)
    resultFile.write('.')
    resultFile.write("\t")
    resultFile.write(healhyText)
    resultFile.write('.')
    resultFile.write("\n")

### Main :

In [81]:
if __name__ == '__main__':

    keyboard = str(choiceConfigKeyboard())
    print(keyboard + " choosed")

    nWord = int(freqWordWithMist())
    jsonToLoad = open(nameOfHealthyCorpus, 'r')
    
    j = 0

    for ligne in jsonToLoad:
        try:
            jsonsLine = json.loads(ligne)
            lId = jsonsLine['id']
            leText = jsonsLine['text']
            if len(leText) < 50 and len(leText) > 5:
                print('Before : ', leText)
                initialText = splitText(leText)
                print(initialText)
                if j == 0:
                    for i in range(1): # nombre de fois que nous passons la phrase dans le générateur de fautes
                        textWithMist = generateErrorInWord(keyboard, nWord, initialText)
                        print('Text with mistake : ', textWithMist, '\n')
                        # leText = " ".join(leText)
                        # saveResult(lId, leText, textWithMist)
                        # nWord += 1
                    # j += 1
                else:
                    saveResult(lId, leText, leText)
                    j += 1
                
                if j == 3:
                    j = 0
                
        except:
            errorFile = open(errorLigneGenCorpus, "a")
            errorFile.write(json.dumps(jsonsLine))
            errorFile.write('\n')
            errorFile.close()
            # print('-----ERROR-----\n', jsonsLine, '\n----- END -----')

    resultFile.close()

    print("\n\nFinish !")

This keyboard config are available :
[0]: azerty-numberPad_hori.txt
[1]: azerty-numberPad_hori-verti-diag.txt
[2]: azerty_hori-verti.txt
[3]: azerty_hori.txt
[4]: azerty-numberPad_hori-verti.txt
[5]: azerty_hori-verti-diag.txt
Thanks to choose keyboard disposition: [default: 0] 
azerty-numberPad_hori.txt choosed

What will be the frequency of words with mistakes ?
[min: 1; max: 9; default: 1] 


Before :  Oscar du cinéma
['Oscar', 'du', 'cinéma']
entrée:  ['Oscar', 'du', 'cinéma']
r :  3
mot en cours :  Oscar
r :  2
mot en cours :  du
r :  2
mot en cours :  cinéma
textToShow :  Oscar du cniéma
Text with mistake :  Oscar du cniéma 

Before :  Démocratie directe
['Démocratie', 'directe']
entrée:  ['Démocratie', 'directe']
r :  4
mot en cours :  Démocratie
r :  4
mot en cours :  directe
textToShow :  Démocratye dyrecte
Text with mistake :  Démocratye dyrecte 

Before :  Guy Parmelin
['Guy', 'Parmelin']
entrée:  ['Guy', 'Parmelin']
r :  3
mot en cours :  Guy
r :  1
mot en cours :  Parmelin